In [ ]:
#  !pip install meteostat
#  !pip install fastquant
#  !pip install feedparser

In [5]:
# imports

import pandas as pd
from datetime import datetime
import os

## Weather
from meteostat import Point, Daily

## NIKL prices
from fastquant import get_pse_data

## feed parsing news - economic news, etc
import feedparser

## import spacy for transformation
import spacy
import ast

## WEATHER

In [2]:
# Set time period
start = datetime(2020, 1, 1)
end = datetime.now()

# Create Point for Cagayan De Oro - Surigao weather station not returning anything
cdo = Point(8.4542, 124.6319)

# Get daily data for 2018
data = Daily(cdo, start, end)
data = data.fetch()

# Save the data to .csv
data.to_csv(f"weather-data-{end.strftime('%m-%d-%Y')}.csv")

## NIKL prices

In [4]:
# get data
data = get_pse_data("JFC", start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))

# save data
data.to_csv(f"NIKL-data-{end.strftime('%m-%d-%Y')}.csv")

190it [01:59,  1.59it/s]


## RSS Feeds

In [6]:
def upload_formatted_rss_feed(feed, feed_name):
    feed = feedparser.parse(feed)
    df = pd.DataFrame(feed['entries'])
    df.to_csv(f"{feed_name}-data-{end.strftime('%m-%d-%Y')}.csv")

In [7]:
upload_formatted_rss_feed("https://businessmirror.com.ph/feed/", "business_mirror")

In [8]:
upload_formatted_rss_feed("https://www.bworldonline.com/feed/", "business_world")

# ACTION PLAN
### Nickel in the PH - mining affected by weather + get business news due to some mines closing temporarily @ height of pandemic
* Weather - meteostat
* NIKL prices - fastquant
* News - RSS feeds of business mirror & business world - perform NER and similar activity as Sprint 3

### Spacy

In [ ]:
# @task(task_id='spacy_ner')
def spacy_ner(ds=None, **kwargs):
    nlp = spacy.load("/model/en_core_web_sm/en_core_web_sm-3.3.0") # import this to the ff. directory
    
    def ner(text):
        doc = nlp(text)
        # print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
        # print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
        ner = {}
        for entity in doc.ents:
            ner[entity.text] = entity.label_
            print(entity.text, entity.label_)
        return ner

    files = os.listdir(DATA_PATH)
    for file in files:
        outfile = f"{DATA_PATH}{file}"
        if not outfile.endswith('.csv'):
            continue
        
        try:
            df = pd.read_csv(outfile)
        
            ################################# TODO: IMPORTANT #########################################
            # you need to find the column where the text/content is located e.g. 'summary' or 'content'
            # and add a conditional logic below
            ###########################################################################################
            if outfile.startswith(f'{DATA_PATH}business_world'):
                df['NER'] = df['content'].apply(lambda x: ner(x))

            else:
                df['NER'] = df['summary'].apply(lambda x: ner(x))

            ###########################################################################################

            df['NER_dict'] = df['NER'].apply(lambda x: ast.literal_eval(str(x)))
            def count_ent(dic):
                empty = {}
                for v in dic.values():
                    if v in empty.keys():
                        empty[v] += 1
                    else:
                        empty[v] = 1 
                return empty
            df["ent_count"] = df['NER_dict'].apply(lambda x: count_ent(x))

            df.to_csv(outfile, index=False)

        except: pass